In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException
import pandas as pd
from pathlib import Path
import unicodedata
import re
class DriverInit:
    ACCEPT_COOKIES_XPATH = "//button[@id='L2AGLb']"
    REVIEWS_TAB_XPATH = "//span[text()='Reviews']"
    GOOGLE_SEARCH_XPATH = "//textarea[@class='gLFyf']"


def __init__(self):
    self.driver = webdriver.Chrome()
    self.wait = WebDriverWait(self.driver, 10)
    self.short_wait = WebDriverWait(self.driver, 5, poll_frequency=0.5)
    self.driver.maximize_window()


def navigate_to_url(self, url):
    self.driver.get(url)
    accept_cookies = self.wait.until(EC.element_to_be_clickable((By.XPATH, self.ACCEPT_COOKIES_XPATH)))
    accept_cookies.click()


def navigate_to_reviews_tab(self):
    reviews_tab = self.wait.until(EC.element_to_be_clickable((By.XPATH, self.REVIEWS_TAB_XPATH)))
    reviews_tab.click()


def close_browser(self):
    self.driver.quit()


def go_to_restaurant_list(self):
    self.wait.until(EC.element_to_be_clickable((By.XPATH, self.GOOGLE_SEARCH_XPATH))).send_keys('restorani')

In [2]:
city='Zagreb'

In [3]:
url = 'https://www.google.hr'
ACCEPT_COOKIES_XPATH = "//button[@id='L2AGLb']"
REVIEWS_TAB_XPATH = "//span[text()='Reviews']"
GOOGLE_SEARCH_XPATH = "//textarea[@class='gLFyf']"
GO_TO_RESTAURANT_LIST_XPATH = "//span[text()='Više mjesta']"

In [4]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
short_wait = WebDriverWait(driver, 5, poll_frequency=0.5)
driver.maximize_window()
driver.get(url)

In [5]:
accept_cookies = wait.until(EC.element_to_be_clickable((By.XPATH, ACCEPT_COOKIES_XPATH)))
accept_cookies.click()

In [6]:
google_search_bar = wait.until(EC.element_to_be_clickable((By.XPATH, GOOGLE_SEARCH_XPATH)))
google_search_bar.send_keys(f'restorani {city}')
google_search_bar.submit()

In [7]:
restaurant_list = wait.until(EC.element_to_be_clickable((By.XPATH, GO_TO_RESTAURANT_LIST_XPATH)))
restaurant_list.click()

In [8]:
MORE_PLACES_XPATH = "//span[text()='Više rezultata']"
LOADING_ICON_XPATH = "//div[@data-hveid='CAQQAw']"
TITLE_XPATH = "//span[@class='OSrXXb e62wic']"

In [9]:
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def get_last_element_text(driver, xpath):
    """
    Retrieves the text of the last element matching the given XPath.
    Returns None if no elements are found.
    """
    try:
        elements = driver.find_elements(By.XPATH, xpath)
        if elements:
            return elements[-1].text
        else:
            return None
    except NoSuchElementException:
        return None

last_unique_element_text = None

def wait_for_loading_complete(driver, loading_icon_xpath, timeout=10):
    """
    Waits for the loading icon to disappear and reappear if necessary,
    indicating that loading sequences have completed.
    """
    end_time = time.time() + timeout
    while time.time() < end_time:
        try:
            # Check for the presence of the loading icon.
            loading_icon_present = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.XPATH, loading_icon_xpath))
            )
            if loading_icon_present:
                # Wait for the loading icon to disappear.
                WebDriverWait(driver, 5).until(
                    EC.invisibility_of_element_located((By.XPATH, loading_icon_xpath))
                )
        except TimeoutException:
            # If the loading icon isn't found or disappears and doesn't reappear, break the loop.
            break




In [10]:
while True:
    try:
        # Wait for the "more places" button to become visible
        more_places_button = wait.until(EC.visibility_of_element_located((By.XPATH, MORE_PLACES_XPATH)))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", more_places_button)

        # Click the button using JavaScript
        driver.execute_script("arguments[0].click();", more_places_button)

        # Dynamically wait for loading to complete
        wait_for_loading_complete(driver, LOADING_ICON_XPATH)

        # Retrieve the text of the last element
        current_last_element_text = get_last_element_text(driver, TITLE_XPATH)

        if current_last_element_text == last_unique_element_text:
            print("No new content loaded. End of list reached.")
            break
        else:
            last_unique_element_text = current_last_element_text

        # Wait for the "more places" button to become visible again
        WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, MORE_PLACES_XPATH)))

    except TimeoutException:
        print("All 'Više mjesta' elements have been clicked, or no more are found.")
        break
    except (ElementNotInteractableException, ElementClickInterceptedException, NoSuchElementException):
        print("The 'Više mjesta' button is no longer interactable or clickable. All content loaded.")
        break

No new content loaded. End of list reached.


In [11]:
titles = [element.text for element in driver.find_elements(By.XPATH, "//span[@class='OSrXXb e62wic']")]

# Print or use the titles as needed
print(titles)

['Restoran Matrix', 'Bota-Šare Restaurant', 'Restoran Šarm', 'Mano2', 'Restoran C6', 'Restoran Marun', 'Restoran HAS', 'Restoran Zagreb', 'Vinodol', "Restoran Zinfandel's", 'Restoran Casablanca', 'Bistro Ribič Zagreb', 'Steakhouse El Toro', 'Restoran "Dubravkin put"', 'TIME Restaurant & Bar', 'Restoran Kvatrić', 'Restoran Nokturno', 'Restorani Rustica - ZAGREB', 'Restoran Vagabund', 'Restoran Sv. Nikola', 'Rougemarin Bistro', 'Restoran Korčula', 'Canzona - Trattoria', 'Sofra - Borovje', 'Pri Zvoncu', 'Gallo', 'Japanski restoran Tekka', 'Restoran Baltazar', 'Gostionica Ficlek', 'Gostionica Purger', 'Restoran Bon Appetit', 'Restoran Apetitlih', 'La Štruk', 'Restoran "Mali Bar"', 'Konoba Bracera', 'Restoran Kaiser', 'Restoran "Kod Pere"', 'Restoran "Zlatna školjka"', 'Kai street food', 'Bistro "Kod Vukušića"', 'Lari & Penati', 'Torero Restaurant', 'Noel', 'Ristorante Carpaccio', 'Restoran Ragusa', 'Restoran "Odeon"', 'Grill Tomislav', 'Trilogija Fino&vino', 'VODNJANKA Zagreb', "O'brok Res

In [12]:
len(titles)

210

In [18]:
import pandas as pd
from pathlib import Path

# Create a DataFrame
df_titles = pd.DataFrame(titles, columns=['Restaurant Title'])

# Define the city variable
city = 'Zagreb'  # Replace with the actual city name as needed

# Specify the output file path relative to the project's root directory
output_file_path = Path(f'../data/{city}/restaurant_list.csv')

# Ensure the directory exists
output_file_path.parent.mkdir(parents=True, exist_ok=True)

# Save to CSV
df_titles.to_csv(output_file_path, index=False)

# Confirm the save
# Print the absolute path of the file for confirmation
print(f'File saved at {output_file_path.resolve()}')


File saved at C:\Users\ivan.zeljeznjak\DataspellProjects\RedditSentimentAnalysis\data\Zagreb\restaurant_list.csv


In [30]:
import pandas as pd
import re
import unicodedata

def clean_name(name):
    # Normalize to handle special characters and convert to lowercase
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode().lower()

    # Remove generic words and handle special cases
    for word in generic_words:
        pattern = r'\b' + re.escape(word) + r'\b'
        name = re.sub(pattern, '', name, flags=re.IGNORECASE)

    # Replace & and - with space to handle cases like "Lari & Penati", "Sofra - Borovje"
    name = name.replace('&', ' ').replace('-', ' ')

    # Remove unwanted characters, keeping spaces and alphanumeric characters
    name = re.sub(r"[^a-z0-9\s]", '', name)

    # Replace multiple spaces with a single space
    name = re.sub(r'\s+', ' ', name)

    # Trim leading and trailing spaces
    name = name.strip()

    # Replace spaces with a single underscore for the file name
    name_for_filename = name.replace(' ', '_')

    # Fix casing for the cleaned title
    cleaned_name = ' '.join(name.split()).title()

    return cleaned_name, name_for_filename

# Apply the enhanced cleaning function
df[['Cleaned Title', 'File Name']] = df.apply(lambda x: pd.Series(clean_name(x['Restaurant Title'])), axis=1)


In [31]:
df

,Restaurant Title,Cleaned Title,File Name
0,Restoran Matrix,Matrix,matrix
1,Bota-Šare Restaurant,Bota Sare,bota_sare
2,Restoran Šarm,Sarm,sarm
3,Mano2,Mano2,mano2
4,Restoran C6,C6,c6
...,...,...,...
205,Brewbites,Brewbites,brewbites
206,Max i Mir Zagreb,Max I Mir Zagreb,max_i_mir_zagreb
207,CHi LE MA Zagreb 吃了吗,Chi Le Ma Zagreb,chi_le_ma_zagreb
208,"Al Papone - grill, pasta & breakfast",Al Papone Pasta Breakfast,al_papone_pasta_breakfast


In [ ]:
#placeholder